In [2]:
from joblib import Parallel, delayed
import ARX_Optimization 
import pandas
import numpy as np
import xlsxwriter
import pathlib
import configparser
from config import input_datadir, result_datadir
import matplotlib.pyplot as plt
import os
import fnmatch

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

In [3]:
def analysis(N, order):
    m, n = order
    row1 = 0
    for filename in os.listdir(input_datadir):
        if filename.endswith('.csv'):
            df = pandas.read_csv(input_datadir/filename)
            ydata = df.T1
            udata = df.Q1
            times = df.Time
            
            # Adding 10 degrees of noise:
            otpts, std_dev = ARX_Optimization.add_noise(ydata, 5) # 5 is the standard deviaton    

            for method in ARX_Optimization.ID_techs:
                if fnmatch.fnmatch(method.__name__, 'DE*'):
                    runtimes = []
                    coefficients = []
                    for i in range(len(otpts)):
                        ni = otpts[i,:]
                        bnds = tuple((-1,1) for j in range(m+n))
                        glob_args = bnds, ni, udata, m, n
                        rts, coeffs = method(*glob_args)
                        runtimes.append(rts)
                        coefficients.append(coeffs)

                else:
                    runtimes = []
                    coefficients = []
                    for i in range(len(otpts)):
                        ni = otpts[i,:]
                        initial_guess = [1 for k in range(m+n)]
                        int_args = initial_guess, ni, udata, m, n
                        rts, coeffs = method(*int_args)
                        runtimes.append(rts)
                        coefficients.append(coeffs)

                    new_filename = os.path.splitext(filename)[0]+str(method_name)+'_m'+str(m)+'_n'+str(n)+"_"+str(N)+'.xlsx'
                    workbook = xlsxwriter.Workbook(result_datadir/new_filename)
                    worksheet1 = workbook.add_worksheet("Input_Data")
                    worksheet2 = workbook.add_worksheet("ID_Results")

                    col = 0
                    for set_ in otpts:
                        row = 0
                        for value, t in zip(set_, times):
                            worksheet1.write(row, 0, t)
                            worksheet1.write(row, col, value)
                            row +=1
                        col+=1
                    row1=0
                    for cf, rt, sig in zip(coefficients, runtimes, std_dev):
                        col1=0
                        for c in cf:
                            worksheet2.write(row1, col1, c)
                            col1+=1
                        worksheet2.write(row1, col1, rt)
                        worksheet2.write(row1, col1+1, sig)
                        row1+=1
                    workbook.close()



In [4]:
Parallel(n_jobs=-1)(delayed(analysis)(x, [1,1]) for x in range(10))

[None, None, None, None, None, None, None, None, None, None]